In [113]:
import numpy as np
import pandas as pd
import seaborn as sns
from   sklearn.compose            import *
from   sklearn.ensemble           import RandomForestClassifier, GradientBoostingClassifier
from   sklearn.impute             import *
from   sklearn.metrics            import confusion_matrix, accuracy_score
from   sklearn.model_selection    import RandomizedSearchCV, train_test_split
from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *

pd.set_option('display.max_colwidth', 1000)

# Import Data

In [115]:
'https://github.com/shahv1057/NFL_4thdown/blob/main/data/play_by_play_2018.csv.gz'
years = [2018,2019,2020] # Look at last three NFL seasons
data = pd.DataFrame()
for year in years:  # Import data for each year
    year_data = pd.read_csv('https://github.com/shahv1057/NFL_4thdown/blob/main/data/'\
                         'play_by_play_' + str(year) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)
    data = data.append(year_data, sort=False)
data.reset_index(drop=True, inplace=True)

# Target Tranformation 
I am trying to answer the question of what play type an NFL team would choose, given a specific pre-play 4th down situation. As a result, I filter the data for only rows that can be considered "pre-play" to avoid any data leakage, and then split the data into my X and y dataframes.

In [116]:
y_classes = ['PASS','RUSH','FIELD_GOAL','PUNT']
data = data[(data['play_type_nfl'].isin(y_classes))&(data['down']==4)]
Xdf = data.drop(['play_type_nfl'],axis=1)
ydf = data['play_type_nfl']

In [117]:
X_train, X_test, y_train, y_test = train_test_split(Xdf, ydf, test_size=.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.2)

In [121]:
def filter_data(X):
    
    """
    Column filtering: Only include features 
    that would be available in a game situation before the play
    begins. 
    Custom feature engineering: Numerically represent
    yard line  and bin score_differential into categorical game
    score situations
    """
    
    Xdf_c = X.copy()
    pre_play_features = [
     'posteam', 
     'defteam',
     'quarter_seconds_remaining',
     'half_seconds_remaining',
     'game_seconds_remaining',
     'game_half',
     'qtr',
     'goal_to_go',
     'yrdln',
     'ydstogo',
     'posteam_timeouts_remaining',
     'defteam_timeouts_remaining',
     'score_differential',
     'season'   
     ]
    Xdf_c = Xdf_c[pre_play_features]
    Xdf_c['ydstogo'] = Xdf_c['ydstogo'].astype(float)
    Xdf_c['score_differential'] = pd.cut(Xdf_c['score_differential'],bins=[-100,-17,-12,-9,-4,0,4,9,12,17,100])
    def convert_yd_line_vars(posteam,ydline):
        if type(ydline)==str:
            newydline = ydline.split()
            if ydline == '50':
                return float(ydline)
            elif posteam == newydline[0]:
                return float(newydline[1])
            else:
                return 100 - float(newydline[1])
        else:
            return np.nan
    Xdf_c['yrdln'] = Xdf_c.apply(lambda x: convert_yd_line_vars(x['posteam'], x['yrdln']), axis=1)
    return Xdf_c

# Model 1 - Random Forest Classifier
Built my Pipeline with three steps:

**1) Column Selection & Transformation**: Apply my filter_data Function Transformer 

**2) Preprocessing**: Impute continuous variables, and impute & one-hot-encode categorical variables

**3) Fit**: Random Forest Classifier

In [126]:
categorical_columns = filter_data(X_train).dtypes != float
con_pipe = Pipeline([('imputer', SimpleImputer(strategy='median', add_indicator=True))
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])
preprocessing = ColumnTransformer([('categorical', cat_pipe,  categorical_columns),
                                   ('continuous',  con_pipe, ~categorical_columns),
                                   ])
pipe_rf = Pipeline([
    ('transform',FunctionTransformer(filter_data,validate=False)),
    ('preprocessing', preprocessing),
    ('rf',RandomForestClassifier(
                                ))
])

### Tune Hyperparameters with Randomized Cross Validation Search

In [128]:
rf_hyperparams = dict(rf__criterion        = ['gini', 'entropy'],      # Loss function to split on
                      rf__min_samples_leaf = np.arange(1, 8, 2),       # Avoid overfitting by stopping splits on low sample leaves
                      rf__max_features     = ["auto", "log2", 'sqrt'], # Number of features choose from to split on at every tree node
                      rf__n_estimators     = [10,50,100]               # Number of Trees, improve generalization
                     )

rf_rand_cv = RandomizedSearchCV(estimator=pipe_rf, 
                                param_distributions=rf_hyperparams, 
                                n_iter=25, 
                                cv=5, 
                                n_jobs=-1,
                                verbose=True,
                                scoring='accuracy')
rf_rand_cv.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('transform',
                                              FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
                                             ('preprocessing',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent')),
                                                                                               ('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                              

### Fit Best Estimator Model with Training Data

In [129]:
pipe_rf = Pipeline([
    ('transform',FunctionTransformer(filter_data,validate=False)),
    ('preprocessing', preprocessing),
    ('rf',rf_rand_cv.best_estimator_['rf'])
])
pipe_rf.fit(X_train,y_train)

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
                ('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  posteam                        True
defteam                        True
quarter_seconds_remaining     False
half_se...
                                                                                 strategy='median'))]),
                                           

### Define Go-For-It metric
Negate the impact of interchanging Pass or Rush to address the real world application of classifying whether or not the team decides to "go for it" on the 4th down play

In [130]:
def go_for_it_metric(y_true,y_pred):
    acc = []
    go_for_it = ['PASS','RUSH']
    return accuracy_score(np.isin(y_true,go_for_it),np.isin(y_pred,go_for_it))

# Random Forest Evaluation
Output confusion matrix, overall accuracy score, class specific accuracy, and "go_for_it" industry-specific metric 

In [131]:
y_pred = pipe_rf.predict(X_valid)
cm = confusion_matrix(y_valid, y_pred, labels=y_classes)
print('---------------------------')
print('Confusion Matrix:')
print(pd.DataFrame(cm,columns=[y+'_PRED' for y in y_classes],index=y_classes))
print ()
print('-----------------------------')
print (f"Overall Accuracy Score: {accuracy_score(y_valid,y_pred)*100:.2f}%",end='\n\n')
print('-----------------------------')
print ("Class Accuracy Scores:",end='\n')
for play_type,acc in (sorted(zip(y_classes,cm.diagonal()/cm.sum(axis=1)),key = lambda x: x[1])):
    print (f"{play_type}: {acc*100:.2f}%")
print ()
print(f'Go-For-It Accuracy Score: {go_for_it_metric(y_valid,y_pred)*100:.2f}%')

---------------------------
Confusion Matrix:
            PASS_PRED  RUSH_PRED  FIELD_GOAL_PRED  PUNT_PRED
PASS               58         21               41         54
RUSH               14         52               10         43
FIELD_GOAL          7          5              403          9
PUNT                4          5               10       1040

-----------------------------
Overall Accuracy Score: 87.44%

-----------------------------
Class Accuracy Scores:
PASS: 33.33%
RUSH: 43.70%
FIELD_GOAL: 95.05%
PUNT: 98.21%

Go-For-It Accuracy Score: 90.48%


# Model 2 - Gradient Boosting Classifier
Built my Pipeline with three steps:

**1) Column Selection & Transformation**: Apply my filter_data Function Transformer 

**2) Preprocessing**: Impute continuous variables, and impute & one-hot-encode categorical variables

**3) Fit**: Gradient Boosting Classifier

In [136]:
categorical_columns = filter_data(X_train).dtypes != float
con_pipe = Pipeline([
                     ('imputer', SimpleImputer(strategy='median', add_indicator=True))
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])
preprocessing = ColumnTransformer([('categorical', cat_pipe,  categorical_columns),
                                   ('continuous',  con_pipe, ~categorical_columns),
                                   ])
pipe_gb = Pipeline([
    ('transform',FunctionTransformer(filter_data,validate=False)),
    ('preprocessing', preprocessing),
    ('gb',GradientBoostingClassifier(
                            ))
])

### Tune Hyperparameters with Randomized Cross Validation Search

In [137]:
gb_hyperparams = dict(gb__learning_rate = [0.15,0.1,0.05,0.01], # How quickly should GB Classifier adjust weights to get to optimal value  
                       gb__n_estimators  = [10,50,100], # Number of trees
                      )

gb_rand_cv = RandomizedSearchCV(estimator=pipe_gb, 
                                param_distributions=gb_hyperparams, 
                                n_iter=12, 
                                cv=5, 
                                n_jobs=-1,
                                verbose=True,
                                scoring='accuracy')
gb_rand_cv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('transform',
                                              FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
                                             ('preprocessing',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent')),
                                                                                               ('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                              

In [138]:
pipe_gb = Pipeline([
    ('transform',FunctionTransformer(filter_data,validate=False)),
    ('preprocessing', preprocessing),
    ('gb',gb_rand_cv.best_estimator_['gb'])
])
pipe_gb.fit(X_train,y_train)

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
                ('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  posteam                        True
defteam                        True
quarter_seconds_remaining     False
half_se...
                                                                   SimpleImputer(add_indicator=True,
                                              

# Gradient Boosting Evaluation
Output confusion matrix, overall accuracy score, class specific accuracy, and "go_for_it" industry-specific metric 

In [139]:
y_pred = pipe_gb.predict(X_valid)
cm = confusion_matrix(y_valid, y_pred, labels=y_classes)
print('---------------------------')
print('Confusion Matrix:')
print(pd.DataFrame(cm,columns=[y+'_PRED' for y in y_classes],index=y_classes))
print ()
print('-----------------------------')
print (f"Overall Accuracy Score: {accuracy_score(y_valid,y_pred)*100:.2f}%",end='\n\n')
print('-----------------------------')
print ("Class Accuracy Scores:",end='\n')
for play_type,acc in (sorted(zip(y_classes,cm.diagonal()/cm.sum(axis=1)),key = lambda x: x[1])):
    print (f"{play_type}: {acc*100:.2f}%")
print('-----------------------------')
print(f'Go-For-It Accuracy Score: {go_for_it_metric(y_valid,y_pred)*100:.2f}%')

---------------------------
Confusion Matrix:
            PASS_PRED  RUSH_PRED  FIELD_GOAL_PRED  PUNT_PRED
PASS               79         21               34         40
RUSH               19         66                5         29
FIELD_GOAL          6         11              399          8
PUNT               12          8                7       1032

-----------------------------
Overall Accuracy Score: 88.74%

-----------------------------
Class Accuracy Scores:
PASS: 45.40%
RUSH: 55.46%
FIELD_GOAL: 94.10%
PUNT: 97.45%
-----------------------------
Go-For-It Accuracy Score: 91.84%


# Final Model
Choose Gradient Boosting Classifier Model as Final Model as it has a higher Overall Accuracy and Go-For-It accuracy on average

In [140]:
final_model = pipe_gb
print (final_model)

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
                ('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  posteam                        True
defteam                        True
quarter_seconds_remaining     False
half_se...
                                                                   SimpleImputer(add_indicator=True,
                                              

In [141]:
final_model.get_params()

{'memory': None,
 'steps': [('transform',
   FunctionTransformer(func=<function filter_data at 0x156d780d0>)),
  ('preprocessing',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('imputer',
                                                     SimpleImputer(add_indicator=True,
                                                                   strategy='most_frequent')),
                                                    ('ohe',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    posteam                        True
   defteam                        True
   quarter_seconds_remaining     False
   half_seconds_remaining        False
   game_seconds_remaining        False
   game_half                      True
   qtr                            True
   goal_to_go                     True
   yrdln                         False
   ydstogo                      

In [142]:
y_pred_final = final_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_final, labels=y_classes)
print('---------------------------')
print('Confusion Matrix:')
print(pd.DataFrame(cm,columns=[y+'_PRED' for y in y_classes],index=y_classes))
print ()
print('-----------------------------')
print (f"Overall Accuracy Score: {accuracy_score(y_test,y_pred_final)*100:.2f}%",end='\n\n')
print('-----------------------------')
print ("Class Accuracy Scores:",end='\n')
for play_type,acc in (sorted(zip(y_classes,cm.diagonal()/cm.sum(axis=1)),key = lambda x: x[1])):
    print (f"{play_type}: {acc*100:.2f}%")
print('-----------------------------')
print(f'Go-For-It Accuracy Score: {go_for_it_metric(y_test,y_pred_final)*100:.2f}%')

---------------------------
Confusion Matrix:
            PASS_PRED  RUSH_PRED  FIELD_GOAL_PRED  PUNT_PRED
PASS              122         32               42         42
RUSH               13         73               15         33
FIELD_GOAL         14         22              482          5
PUNT               16         13                5       1291

-----------------------------
Overall Accuracy Score: 88.65%

-----------------------------
Class Accuracy Scores:
PASS: 51.26%
RUSH: 54.48%
FIELD_GOAL: 92.16%
PUNT: 97.43%
-----------------------------
Go-For-It Accuracy Score: 91.13%


# Final Model Interpretation

    My model performed similarly on the test set as it did during training, ultimately achieving just under 90% accuracy on the test set, which is consistent with my validation results and allows me to conclude that the model is working as expected. This accuracy varied heavily by class in both validation and test predictions, with upwards of 90% of FIELD_GOAL and PUNT plays correctly predicted, but less than 60% of PASS and RUSH plays. These discrepencies makes sense in a game context, given the much more unpredictable nature of PASS and RUSH plays on 4th down. 

    The model was able to capture a lot of the nuances of play selection in 4th down situations. In the context of a game, this type of model could be especially useful for coaches in predicting play selection of an opposing team in critical 4th down situations. Additionally, further analysis of the model features importances could lead to new insights into play tendencies and potentially key indicators of play selection.

# Next Steps

1) Look into more features beyond dataset, such individual players or weather (temp, humidity, rain, snow, wind, etc.) 

2) Test out Deep Learning models to explore potentially better results